# Exploring eBay Car Sales Data

## Introduction

In this project, we'll be working with a dataset of used cars from eBay Kleinanzeigen, a classifieds section of the German eBay website. The aim of this project is to clean and analyse the used car listings using the NumPy and pandas libraries.

In [ ]:
import pandas as pd
import numpy as np

autos = pd.read_csv("autos.csv")

In [ ]:
autos

In [ ]:
autos.info()

In [ ]:
autos.head()

Our dataset has 50,000 rows and 20 columns. Most columns are stored as strings, with some stored as integers.

## Cleaning Column Names

The column names use camel case instead of snake case, so we can't just replace spaces with underscores. Let's convert the column names to snake case and make some of them more descriptive.

In [ ]:
autos.columns

In [ ]:
autos.columns = ['date_crawled', 'name', 'seller', 'offer_type', 'price', 'ab_test',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'num_pics', 'postal_code',
       'last_seen']

In [ ]:
autos.head(1)

## Initial Exploration and Cleaning

Initially we will look for columns which have most values the same, as we cannot get any useful information from these. We will also look for numerical columns which are stored as text which can be cleaned and converted to a numeric type.

In [ ]:
autos.describe(include="all")

The `seller` and `offer_type` columns have almost all values the same (49,999 out of 50,000 are the same). 
The `num_pics` column has the same mean, min, max and quartiles, indicating all values may be the same. Let's investigate whether this is the case:

In [ ]:
autos["num_pics"].value_counts()

Therefore we will remove the `seller`, `offer_type` and `num_pics` columns:

In [ ]:
autos = autos.drop(['seller', 'offer_type', 'num_pics'], axis=1)
autos.head(1)

We also notice that the `price` and `odometer` columns contain numerical data stored as text, so let's fix this:

In [ ]:
autos["price"] = autos["price"].str.replace("$","").str.replace(",","").astype(int)
autos["odometer"] = autos["odometer"].str.replace("km","").str.replace(",","").astype(int)
autos.rename({"odometer":"odometer_km"},axis=1,inplace=True)

In [ ]:
autos.head()

## Exploring the Odometer and Price Columns

Let's now explore the `odometer_km` and `price` columns, looking for data that doesn't look quite right, including outliers.

In [ ]:
autos["odometer_km"].unique().shape

In [ ]:
autos["odometer_km"].describe()

In [ ]:
autos["odometer_km"].value_counts()

For the `odometer_km` column we observe that there are only discrete values, indicating that users may have had to choose from a predetermined number of values. The values in this column are left-skewed: there are much more high mileage vehicles.

In [ ]:
autos["price"].unique().shape

In [ ]:
autos["price"].describe()

In [ ]:
autos["price"].value_counts().head(10)

The prices are also rounded here, but there are much more values so this indicates that users may have rounded the values at their own discretion. There are 1,421 cars with a price of \\$0. The maximum price is \\$100,000,000 which seems like an outlier. Let's explore these high prices further:

In [ ]:
autos["price"].value_counts().sort_index(ascending=False).head(20)

In [ ]:
autos["price"].value_counts().sort_index(ascending=True).head(20)

It seems that prices steadily increase up to \\$350,000 but increase rapidly from then. So we will discard prices over \\$350,000. Since eBay is an auctioning site, it isn't necessarily unexpected to see extremely low prices, so we will keep these, except for \\$0.

In [ ]:
autos = autos[autos["price"].between(1,350001)]
autos.describe()

## Exploring the Date Columns

There are 5 date columns, some from the website itself, and some added by the crawler:

- `date_crawled`: added by the crawler
- `last_seen`: added by the crawler
- `ad_created`: from the website
- `registration_month`: from the website
- `registration_year`: from the website

In [ ]:
autos.info()

`registration_month` and `registration_year` are stored numerically, so can be easily processed. However, `date_crawled`, `ad_created` and `last_seen` are stored as strings so we need to do some manipulation first.

In [ ]:
autos[['date_crawled','ad_created','last_seen']][0:5]

Let's calculate the distribution of the values in these columns, as percentages:

In [ ]:
autos['date_crawled'].str[:10].value_counts(normalize=True,dropna=False).sort_index(ascending=True)

In [ ]:
autos['date_crawled'].str[:10].value_counts(normalize=True,dropna=False).sort_values()

It seems that the data was crawled daily over the period 5th March 2016 to 3rd April 2016, with about the same number of listings crawled each day.

In [ ]:
autos['ad_created'].str[:10].value_counts(normalize=True,dropna=False).sort_index(ascending=True)

There is a large range of ad created dates, ranging from June 2015 to April 2016.

In [ ]:
autos['last_seen'].str[:10].value_counts(normalize=True,dropna=False).sort_index()

The crawler recorded the date it last saw a listing, allowing us to determine when a listing was removed, most likely due to the car being sold.

The last three days contain a disproportionate amount of last seen values. Given that these are more than 6 times the values from the previous days, it's unlikely that there was a massive spike in sales, and more likely that these values are to do with the crawling period ending.

Let's try to understand the distribution of `registration_year`:

In [ ]:
autos["registration_year"].describe()

The minimum of `1000` and the maximum of `9999` are clear outliers.

## Dealing with Incorrect Registration Year Data

There were some clear outliers that we just spotted. A car can't be registered after it was listed, so a registration year above 2016 is definitely inaccurate. The earliest registration year is likely in the early 1900s.

In [ ]:
autos["registration_year"].value_counts(normalize=True,dropna=False).sort_index()

In [ ]:
autos["registration_year"].sort_values(ascending=True).tail(50)

2016 is the highest acceptable value because cars can't have been registered after they were listed. The lowest value we accept is 1900 because cars were not mass-produced before then.

In [ ]:
autos= autos[autos["registration_year"].between(1900,2016)]
autos["registration_year"].value_counts(normalize=True).head(10)

In [ ]:
autos["registration_year"].describe()

It seems that most cars were registered in the late 1990s / early 2000s.

## Exploring Price by Brand

In [ ]:
autos["brand"].unique().shape[0]

In [ ]:
100*autos["brand"].value_counts(normalize=True)

Volkswagen makes up 21% of the cars listed, with exclusively German brands occupying the top 5.
Now let's explore how prices vary according to brand, limiting our scope to the top 10 brands.

In [ ]:
brand_counts = autos["brand"].value_counts(normalize=True)
#common_brands = brand_counts[brand_counts > .05].index # if you want to choose the brands that have 5% over the total
common_brands = brand_counts.head(10).index # the top 10 brands
print(common_brands)

brand_mean_prices = {}

for brand in common_brands:
    brand_only = autos[autos["brand"] == brand]
    mean_price = brand_only["price"].mean()
    brand_mean_prices[brand] = int(mean_price)

brand_mean_prices

Of the top 10 brands:

- Mercedes-Benz, Audi and BMW are the most expensive
- Opel is the least expensive
- Volkswagen, the most popular brand, is somewhere in the middle in terms of price.

## Storing Aggregate Data in a DataFrame

We noticed a distinct price gap between the top car brands. Let's now explore whether this is linked to mileage by using aggregation. 

We will make series objects from average price and mileage. We can combine the data from both series objects into a single dataframe (with a shared index) and display the dataframe directly. To do this, we'll use two pandas methods: 

- pandas series constructor
- pandas dataframe constructor

Let's first use aggregation to find the average mileage, like we did for price:

In [ ]:
brand_mean_mileage = {}

for brand in common_brands:
    brand_only = autos[autos["brand"] == brand]
    mean_mileage = brand_only["odometer_km"].mean()
    brand_mean_mileage[brand] = int(mean_mileage)

Now let's use the series and dataframe constructors:

In [ ]:
mean_mileage = pd.Series(brand_mean_mileage).sort_values(ascending=False)
mean_prices = pd.Series(brand_mean_prices).sort_values(ascending=False)

In [ ]:
brand_info = pd.DataFrame(mean_mileage,columns=["mean_mileage"])
brand_info

Let's add the price as another column:

In [ ]:
brand_info["mean_price"] = mean_prices
brand_info

The mileages do not vary as much as the prices, although there is a generally positive correlation between mileage and price.

# Summary

- We explored the dataset, cleaning some incorrect data, and changed our column labels from camelCase to snake_case.

- We removed non-numeric characters in columns representing numerical data, editing our column label as appropriate.

- We removed outliers in the `price` and `odometer` columns.

- We explored the `registration_year` to remove incorrect data, such as cars registered after being listed, or before cars were even in mass production.

- Finally, we used aggregation to filter the cars by mean price and mileage for the top brands and found a slight correlation between mileage and price for these brands. Prices displayed more variation than mileage.